## Load in data and remove extra columns ##

In [718]:
import pandas as pd
import glob
import os

from datetime import datetime

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

for file in csv_files:
    df = pd.read_csv(file)

historic = False
if 'Date' in df:
    historic = True
    df = df[['Date', 'Last', 'First', 'Score', 'Sleep Hours']]
else:
    df = df[['Last', 'First', 'Readiness Score', 'Sleep']]

df.head()

KeyError: "['Last' 'First' 'Readiness Score' 'Sleep'] not in index"

## Add rows with week averages ##

In [ ]:
if historic:
    dates = df['Date'].unique()
    weeks = []
    week = []

    # Separate days into weeks
    for idx, date in enumerate(dates):
        
        # Create datetime object from str
        clean_date = datetime.strptime(date, "%Y-%m-%d")
        
        # Add day to current week
        week.append(date)
        
        # End of week (Monday)
        if clean_date.weekday() == 0 or idx == len(dates) - 1:
           weeks.append(week)
           week = []

    # Add row with team averages for each full week
    for week in weeks:
        score_mean = df.loc[df['Date'].isin(week), 'Score'].mean()
        sleep_mean = df.loc[df['Date'].isin(week), 'Sleep Hours'].mean()
        row = ({
            'Date': f"{week[0]} Weekly Avg)",
            'Last': ' Team',
            'First': 'Average',
            'Score': score_mean,
            'Sleep Hours': sleep_mean
        })
        idx = df.loc[df['Date'] == week[-1]].index.array[-1] + 1
        df2 = pd.DataFrame(row, index=[idx])
        df = pd.concat([df, df2]).sort_index().reset_index(drop=True)
    df['Score'] = df['Score'].round(2)
    df['Sleep Hours'] = df['Sleep Hours'].round(1)
df.head(60)
    



,Date,Last,First,Score,Sleep Hours
0,2023-03-22,Allen,Emmie,95.75,8.0
1,2023-03-22,Bell,Maycee,93.75,9.0
2,2023-03-22,Cheesman,Sydney,75.00,9.0
3,2023-03-22,Colton,Emily,71.59,10.0
4,2023-03-22,Cox,Isabel,52.85,7.0
5,2023-03-22,Dahlien,Maddie,55.69,9.0
6,2023-03-22,Dellaperuta,Tori,70.27,8.0
7,2023-03-22,Dellarose,Tessa,90.25,8.0
8,2023-03-22,Dominguez,Makenna,46.17,7.0
9,2023-03-22,Elgin,Emerson,72.37,9.0


## Add conditional formatting and export to excel ##

In [ ]:
if historic:
    score = 'Score'
    sleep = 'Sleep Hours'

    # Create multiindex dataframe
    df = df.pivot_table(index=['Last', 'First'], columns=['Date'], values=['Score', 'Sleep Hours']).sort_index(axis=1, level='Date', ascending=False)

    # Create individual dataframes for each week
    wks = []
    wk = pd.DataFrame()
    for idx, col in enumerate(df):
        if idx > 0 and col[0] == 'Sleep Hours' and 'W' in col[1]:
            wk = wk.iloc[:, ::-1]
            wks.append(wk)
            wk = pd.DataFrame()
        wk[col] = df[col]
else:
    score = 'Readiness Score'
    sleep = 'Sleep'


# Date conditional formatting
if historic:
    def date(df):
        highlight = ''
        if int(df['Date'][-1]) % 2 == 0:
            highlight = 'background-color: #D59CFF'
        else:
            highlight = 'background-color: #EFDAFF'
        
        if df['Date'][0] == 'W':
            highlight = 'background-color: #7BAFD4'
        return [highlight]

# Score conditional formatting
def score_f(val):
    highlight = ''
    if val < 60:
        highlight = 'background-color: red'
    elif val >= 60 and val < 70:
        highlight = 'background-color: orange'
    elif val >= 70 and val < 85:
        highlight = 'background-color: yellow'
    elif val >= 85:
        highlight =  'background-color: #15DE00'
    return highlight

# Sleep conditional formatting
def sleep_f(val):
    highlight = ''
    if val < 6:
        highlight = 'background-color: red'
    elif val >= 6 and val < 8:
        highlight = 'background-color: yellow'
    elif val >= 8:
        highlight =  'background-color: #15DE00'
    return highlight

# Blank cells
def blank(val):
    highlight = ''
    if val == 'nan':
        highlight = 'backgroundcolor: white'
    return highlight

new_df = (df
    .style
    .apply(score_f, subset=[score], axis=1)
    .apply(sleep_f, subset=[sleep], axis=1)
    )

if historic:
    new_df = (new_df.apply(date, subset=['Date'], axis=1))
    
    with pd.ExcelWriter('WSoccer FitFor90 Readiness.xlsx', engine='xlsxwriter') as writer:
        for idx, w in enumerate(wks):
            print()
            styled_w = (w
            .style
            .applymap(score_f, subset=[col for col in w.columns if col[0] == 'Score'])
            .applymap(sleep_f, subset=[col for col in w.columns if col[0] == 'Sleep Hours'])
            .applymap(blank)
            )
            styled_w.to_excel(writer, freeze_panes=(0, 2), sheet_name=str(w.columns[0][1]))

            workbook  = writer.book
            worksheet = writer.sheets[str(w.columns[0][1])]
            worksheet.set_column(0, len(w), 25)

else:
    new_df.to_excel('WSoccer_Readiness.xlsx')

%load_ext watermark
%watermark -v -m -p pandas,openpyxl,xlsxwriter

